In [ ]:
import sys, os

# Expand the `~` to the full path and append it to `sys.path`
full_path = os.path.expanduser("~/Documents/github/aliyun-devops")
sys.path.append(full_path)

from odps_client import get_odps_sql_result_as_df
from datetime import datetime, timedelta

import pandas as pd

ds = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")

# Create a directory for the markdown files if it doesn't exist
output_dir = f"output_crm_followup_{ds}"
os.makedirs(output_dir, exist_ok=True)

## azure OpenAI方法定义

In [ ]:
# Import the base64 encoding library.
import base64, os, time
import logging

# Configure the logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

proxy_object = {"http": "http://127.0.0.1:8001", "https": "http://127.0.0.1:8001"}


from openai import AzureOpenAI

client_gpt4o = AzureOpenAI(
    api_version="2024-03-01-preview",
    azure_endpoint="https://xm-ai-us2.openai.azure.com",
    api_key=os.getenv("AZURE_GPT4O_API_KEY", ""),
)

client_gpt4o_mini = AzureOpenAI(
    api_version="2024-03-01-preview",
    azure_endpoint="https://xm-ai-us.openai.azure.com",
    api_key=os.getenv("AZURE_GPT4O_MINI_API_KEY", ""),
)


def call_azure_openai(
    messages=[], retrying=1, is_gpt4o=False, is_json=True, max_tokens=16384
) -> (str, bool):
    if retrying < 0:
        return "超过了最大重试次数", False
    completion = None
    ## gpt3.5:  gpt-35-turbo-16k,
    ## got4o:   gpt-4o
    ## got4o-mini:   gpt-4o-mini
    model = "gpt-4o-mini"
    client_to_use = client_gpt4o_mini
    if is_gpt4o:
        logging.info(f"using GPT-4o...:{messages}")
        model = "gpt-4o"
        client_to_use = client_gpt4o
    try:
        completion = client_to_use.chat.completions.create(
            model=model,
            temperature=0.1,
            max_tokens=max_tokens,
            messages=messages,
            response_format={"type": "json_object"} if is_json else {"type": "text"},
        )
        response = completion.choices[0].message.content
        if (
            len(completion.choices) <= 0
            or f"{completion.choices[0].finish_reason}" == "content_filter"
        ):
            return f"azure过滤了本次请求:{completion.choices[0].to_dict()}", False
        if response is None:
            logging.info(f"azure API返回了异常:{completion.to_dict()}")
            time.sleep(10)
            return call_azure_openai(
                messages=messages,
                retrying=retrying - 1,
                is_gpt4o=is_gpt4o,
            )
        logging.info(f"total usage:{completion.usage}")
        return response, True
    except Exception as e:
        logging.info(
            f"请求azure接口报错了:{e}\n messages:{messages}, completion:{completion}"
        )
        if retrying <= 0 or "Error code: 400" in f"{e}":
            return f"{e}", False
        logging.info(f"重试中...{retrying}, messages:{messages}")
        return call_azure_openai(
            messages=messages,
            retrying=retrying - 1,
            is_gpt4o=is_gpt4o,
        )

## 先获取ODPS数据：summerfarm_ds.app_sales_task_trade_performance_df

In [ ]:
task_name='8月复活任务'
sql=f"""
select 
    task_id as 任务ID,
    task_name as 任务名称,
    cust_id as 客户ID,
    cust_name as 客户名称,
    cust_type as 客户业态,
    bd_id as 最新归属BD_ID,
    bd_name as 最新归属BD名字,
    m1 as 最新M1,
    m2 as 最新M2,
    m3 as 最新M3,
    zone_name as 销售区域,
    visit_bd as 实际拜访BD,
    add_time as 拜访时间,
    condition as 拜访记录,
    ds
from summerfarm_ds.app_sales_task_trade_performance_df
where ds=max_pt('summerfarm_ds.app_sales_task_trade_performance_df')
and task_name like '{task_name}%'
and condition is not null
"""

task_visit_df=get_odps_sql_result_as_df(sql=sql)
task_visit_df.head(4)

In [ ]:
def extract_visit_info(visit_text='是否见到核心KP-#否#；客户长时间未下单原因-品质差不稳定；本次拜访前做了什么准备-了解客户下单品；向客户推荐什么活动及商品-月活卷；客户主要采买渠道-渠道商；不愿当场下单原因-晚点看一下；竞对信息-#渠道商#；其他-催单'):
    if len(visit_text) < 1:
        return {}
    import json
    result = {}
    try:
        visit_text=visit_text.split('；')
        for text in visit_text:
            key_value=text.split('-')
            if len(key_value) < 2:
                result[key_value[0]]='无'
            else:
                result[key_value[0]]=key_value[1]
        return result
    except Exception as e:
        print(f"解析JSON失败: {e}")
    return result

task_visit_df['拜访记录JSON']=task_visit_df['拜访记录'].apply(extract_visit_info)
key_columns=['客户主要采买渠道',
'客户长时间未下单原因',
'向客户推荐什么活动及商品',
'不愿当场下单原因',
'其他',]

for key in key_columns:
    task_visit_df[key]=task_visit_df['拜访记录JSON'].apply(lambda x: x.get(key,'无'))

task_visit_df.head(4)

In [ ]:
# Generate new record id for each row, starting from 100 and increasing by 1 for each row
task_visit_df['记录ID'] = range(100, 100 + len(task_visit_df))
task_visit_df.head(4)


In [ ]:
system_prompt = f"""你是一位经验丰富的销售管理专家,负责评估销售团队的客户拜访记录质量。你需要根据以下标准来判断每条拜访记录是否有效:

1. 是否收集到有效的"客户长时间未下单原因":
   - 记录中应该明确指出客户未下单的具体原因,如价格、产品质量、竞品等。
   - 原因应该具体而不是笼统,例如"客户不满意"这样的描述是不够的。
   - 良好的记录应该包含客户的直接反馈或销售人员的专业分析。

2. 是否有"向客户推荐什么活动及商品":
   - 记录中应该明确列出向客户推荐的具体活动或商品名称。
   - 推荐应该与客户的需求或未下单原因相关联。
   - 优秀的记录会包含推荐的理由或预期效果。

3. 是否收集到有效的"不愿当场下单原因":
   - 记录应该清楚说明客户为什么不愿意当场下单。
   - 原因应该是具体的,如预算限制、需要内部讨论等。
   - 高质量的记录会包含针对这些原因的后续跟进计划。

4. 整体评估:
   - 记录的信息是否完整、清晰、有逻辑性。
   - 是否包含actionable insights(可执行的洞察)。
   - 记录是否反映出销售人员的专业性和对客户需求的理解。

请根据以上标准,对每条拜访记录进行评估,并给出"有效"或"无效"的判断。如果判断为"无效",请指出具体的不足之处和改进建议。你的评估将用于指导销售团队改进他们的客户拜访质量和记录方式。
用户会给你一个CSV文件内容作为输入，CSV包含2列：记录ID,拜访记录。请你以**JOSN**的格式输出，输出的JSON应该包含3个字段：记录ID,是否有效,评论。

以下是一个用户输入样例：
```csv
记录ID,拜访记录
1001,是否见到核心KP-#否#；客户长时间未下单原因-联系不上；本次拜访前做了什么准备-历史品；向客户推荐什么活动及商品-金钻；客户主要采买渠道-批发市场；不愿当场下单原因-联系不上；客户生意-其他；竞对信息-#批发市场,渠道商#；其他-联系不上
1002,竞对信息-#批发市场#
```


以下是一个合格的输出：
{{"records":[{{"记录ID":1001,"是否有效":"有效","评论":"包含了客户未下单原因（库存未消化），推荐了活动和商品（金钻），提到了不愿当场下单的原因（联系不上）"}},
{{"记录ID":1002,"是否有效":"无效","评论":"信息严重不足，仅有一个标签，没有任何实质性的拜访记录内容"}}]}}
"""


def call_ai_api_to_get_extract_visit_info(visit_text=""):
    json_text, is_ok = call_azure_openai(
        is_gpt4o=False,
        is_json=True,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": f"{system_prompt}",
                    }
                ],
            },
            {
                "role": "user",
                "content": [{"type": "text", "text": visit_text}],
            },
        ],
    )

    logging.info(f"json_text:{json_text}, visit_text:{visit_text}")
    return json_text


from datetime import datetime

date_of_now = datetime.now().strftime("%Y-%m-%d")

In [ ]:
task_visit_ai_df = task_visit_df[["记录ID", "拜访记录"]].copy()

import pandas as pd
import json

results = []

batch_size = 10
for i in range(0, len(task_visit_ai_df), batch_size):
    batch_df = task_visit_ai_df.iloc[i : i + batch_size]
    csv_content = batch_df.to_csv(index=False, header=True)
    json_result = call_ai_api_to_get_extract_visit_info(csv_content)
    try:
        results.extend(json.loads(json_result)["records"])
    except Exception as e:
        print(f"解析JSON失败: {e}, json_result:{json_result}")

# Convert the results array to a DataFrame
results_df = pd.DataFrame(results)

# Merge the results DataFrame with the original DataFrame
merged_df = pd.merge(task_visit_ai_df, results_df, on="记录ID", how="left")

merged_df.to_csv(f"{output_dir}/merged_df.csv", index=False)
merged_df.head(10)

In [ ]:
task_name=task_visit_df.iloc[0]['任务名称']

In [ ]:
task_visit_with_result_df=pd.merge(task_visit_df,merged_df,on='记录ID',how='left',suffixes=('', '_y'))
task_visit_with_result_df.to_csv(f"{output_dir}/{task_name}_AI评论结果.csv", index=False)

In [ ]:
dont_order_reasons = task_visit_with_result_df.groupby("不愿当场下单原因").agg(
    count=("记录ID", "count")
)
dont_order_reasons_df = dont_order_reasons.reset_index()[["不愿当场下单原因", "count"]]
dont_order_reasons_df.sort_values(by="count", ascending=False, inplace=True)
dont_order_reasons_df.rename(columns={"count": "出现次数"}, inplace=True)
dont_order_reasons_csv = dont_order_reasons_df.to_csv(index=False)
dont_order_reasons_csv

In [ ]:
longtime_dont_order_reasons = task_visit_with_result_df.groupby(
    "客户长时间未下单原因"
).agg(count=("记录ID", "count"))
longtime_dont_order_reasons = longtime_dont_order_reasons.reset_index()[
    ["客户长时间未下单原因", "count"]
]
longtime_dont_order_reasons.rename(columns={"count": "出现次数"}, inplace=True)
longtime_dont_order_reasons.sort_values(by="出现次数", ascending=False, inplace=True)
longtime_dont_order_reasons_csv = longtime_dont_order_reasons.to_csv(index=False)
longtime_dont_order_reasons_csv

In [ ]:
reasons_long_delays_customer_orders_system_prompt = """# 客户未下单原因分类系统

你是一个专门用于分类客户未下单原因的 AI 助手。你的任务是将销售员收集到的各种客户未下单的原因归类到预定义的类别中。分类过程分为两个层次：主要类别和二级分类。

## 分类规则

1. 每个原因都必须被归类到一个主要类别和一个二级分类。
2. 如果一个原因可能属于多个类别，选择最相关或最突出的一个。
3. 使用提供的类别和分类，不要创建新的类别。
4. 如果实在无法归类，使用"其他"类别。

## 主要类别和二级分类

1. 价格因素
   - 1.1 价格过高
   - 1.2 缺乏价格竞争力
   - 1.3 价格上涨

2. 需求量减少
   - 2.1 正常用量减少
   - 2.2 季节性需求减少
   - 2.3 业务规模缩小

3. 经营状况不佳
   - 3.1 生意冷淡
   - 3.2 财务困难
   - 3.3 市场竞争加剧

4. 库存充足
   - 4.1 库存未用完
   - 4.2 过度采购
   - 4.3 销售周期延长

5. 暂时性因素
   - 5.1 假期或节日
   - 5.2 短期业务调整
   - 5.3 临时停业

6. 供应商更换
   - 6.1 找到新供应商
   - 6.2 与其他供应商长期合作
   - 6.3 供应商多样化策略

7. 经营状况变化
   - 7.1 业务转型或调整
   - 7.2 停业或倒闭
   - 7.3 所有权或管理层变更

8. 产品质量问题
   - 8.1 质量不稳定
   - 8.2 不满意过往产品
   - 8.3 竞品质量更好

9. 物流因素
   - 9.1 配送不便
   - 9.2 运费问题
   - 9.3 交货时间长

10. 其他因素
    - 10.1 公司政策限制
    - 10.2 沟通问题
    - 10.3 未明确说明原因

## 输入格式：CSV

### 输入示例
原因
价格太贵了，买不起
最近生意不好，用的很少

## 输出格式：JSON

对于每个输入的原因，请按以下格式输出：

{"原因": "[原始原因]","主要类别": "[类别名称]","二级分类": "[分类名称]"}

### 输出示例

{"records":[{"原因": "价格太贵了，买不起","主要类别": "价格因素","二级分类": "价格过高"},{"原因": "最近生意不好，用的很少","主要类别": "经营状况不佳","二级分类": "生意冷淡"}]}

现在，你已经准备好开始分类客户未下单的原因了。请等待输入，然后按照上述格式进行分类。"""

def call_ai_to_cluster_reasons_long_delays_customer_orders(visit_text=""):
    json_text, is_ok = call_azure_openai(
        is_gpt4o=False,
        is_json=True,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": f"{reasons_long_delays_customer_orders_system_prompt}",
                    }
                ],
            },
            {
                "role": "user",
                "content": [{"type": "text", "text": visit_text}],
            },
        ],
    )

    logging.info(f"json_text:{json_text}, visit_text:{visit_text}")
    return json_text

In [ ]:
results = []

batch_size = 50
for i in range(0, len(longtime_dont_order_reasons), batch_size):
    batch_df = longtime_dont_order_reasons.iloc[i : i + batch_size]
    csv_content = batch_df.to_csv(index=False, header=True)
    json_result = call_ai_to_cluster_reasons_long_delays_customer_orders(csv_content)
    try:
        results.extend(json.loads(json_result)["records"])
    except Exception as e:
        print(f"解析JSON失败: {e}, json_result:{json_result}")

# Convert the results array to a DataFrame
reasons_long_delays_customer_order_df = pd.DataFrame(results)
reasons_long_delays_customer_order_df.head(10)


In [ ]:
reasons_long_delays_customer_order_df.to_csv(f"{output_dir}/{task_name}_AI评论结果_客户长时间未下单原因归类.csv", index=False)

### 不愿意当场下单原因分类

In [ ]:
no_order_reasons_system_prompt = """# 客户不下单原因分类助手

你是一个专门用于分类客户不下单原因的AI助手。你的任务是分析销售员收集到的客户反馈,并将每条反馈准确地归类到预定义的分类体系中。

## 分类体系

将每条客户反馈归类到以下10个主要分类之一,然后进一步归类到相应的二级分类:

1. 暂时不需要
   - 还有库存/货
   - 用量少
   - 刚进货/买过/已下单

2. 价格因素
   - 价格太贵/高
   - 需要对比价格
   - 等待优惠/活动

3. 决策延迟
   - 考虑中
   - 晚点看看
   - 需要时再下单

4. 采购流程限制  
   - 负责人不在
   - 公司管控
   - 需要审批

5. 经营状况影响
   - 生意不好
   - 用量减少
   - 暂停营业

6. 季节性因素
   - 学校放假
   - 店铺休假
   - 节假日影响

7. 供应商相关
   - 更换供应商
   - 有其他渠道
   - 公司统一采购

8. 产品/服务问题
   - 质量不稳定
   - 缺货严重
   - 品类不满足需求

9. 店铺变动
   - 店铺装修
   - 店铺转让
   - 搬迁/关店

10. 沟通障碍
    - 联系不上
    - 没有回复
    - 停机/关机

## 处理流程

1. 仔细阅读每条客户反馈。
2. 根据反馈内容,确定最适合的主要分类。
3. 在选定的主要分类下,进一步确定最合适的二级分类。
4. 如果一条反馈可能属于多个分类,选择最主要或最直接相关的分类。
5. 如果遇到难以归类的反馈,尝试理解客户的潜在意图,选择最接近的分类。

## 输入格式：CSV

### 输入示例
原因,出现次数
暂时不需要,212
已下单,187

## 输出格式

对于每条客户反馈,请按以下JSON格式输出:

{"原因": "[原始原因]","主要类别": "[类别名称]","二级分类": "[分类名称]"}

### 输出示例
{"records":[{"原因": "暂时不需要","主要类别": "暂时不需要","二级分类": "还有库存/货"},{"原因": "已下单","主要类别": "暂时不需要","二级分类": "刚进货/买过/已下单"}]}

## 注意事项

- 保持客观性,避免对客户反馈做主观判断。
- 如果一条反馈确实无法归类,标记为"无法归类"并简要说明原因。
- 对于模糊不清的反馈,基于现有信息做出最佳判断。
- 如果发现反复出现的新模式,可以建议添加新的分类或调整现有分类。

请基于以上指南,准确高效地对客户不下单原因进行分类。"""


def call_ai_to_cluster_no_order_reasons(visit_text=""):
    json_text, is_ok = call_azure_openai(
        is_gpt4o=False,
        is_json=True,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": f"{no_order_reasons_system_prompt}",
                    }
                ],
            },
            {
                "role": "user",
                "content": [{"type": "text", "text": visit_text}],
            },
        ],
    )

    logging.info(f"json_text:{json_text}, visit_text:{visit_text}")
    return json_text

In [ ]:
no_order_results = []

for i in range(0, len(dont_order_reasons_df), batch_size):
    batch_df = dont_order_reasons_df.iloc[i : i + batch_size]
    csv_content = batch_df.to_csv(index=False, header=True)
    json_result = call_ai_to_cluster_no_order_reasons(csv_content)
    try:
        no_order_results.extend(json.loads(json_result)["records"])
    except Exception as e:
        print(f"解析JSON失败: {e}, json_result:{json_result}")

# Convert the results array to a DataFrame
no_order_reasons_df = pd.DataFrame(no_order_results)
no_order_reasons_df.to_csv(f"{output_dir}/{task_name}_AI评论结果_不立刻下单原因归类.csv", index=False)

In [ ]:
print(task_visit_with_result_df.columns)
print("no_order_reasons_df",no_order_reasons_df.columns)
print("reasons_long_delays_customer_order_df",reasons_long_delays_customer_order_df.columns)

In [ ]:
import pandasql

task_visit_with_result_str_df=task_visit_with_result_df.astype(str)

task_visit_with_result_str_df=pandasql.sqldf("""
select 
    a.任务id, 
    a.任务名称, 
    a.客户id, 
    a.客户名称, 
    a.客户业态, 
    a.最新归属bd_id, 
    a.最新归属bd名字, 
    a.最新m1,
    a.最新m2, 
    a.最新m3, 
    a.销售区域, 
    a.实际拜访bd, 
    a.拜访时间, 
    a.拜访记录, 
    a.拜访记录JSON,
    a.客户主要采买渠道, 
    a.客户长时间未下单原因, 
    a.向客户推荐什么活动及商品, 
    a.不愿当场下单原因, 
    a.其他, 
    a.是否有效, 
    a.评论,
    b.主要类别 as 不立刻下单原因主要类别,
    b.二级分类 as 不立刻下单原因二级分类,
    c.主要类别 as 客户长时间未下单原因主要类别,
    c.二级分类 as 客户长时间未下单原因二级分类
from task_visit_with_result_str_df a
left join no_order_reasons_df b on a.不愿当场下单原因=b.原因
left join reasons_long_delays_customer_order_df c on a.客户长时间未下单原因=c.原因    
""")

task_visit_with_result_str_df.to_csv(f"{output_dir}/{task_name}_AI评论结果_合并.csv", index=False)
task_visit_with_result_str_df.head(20)


In [ ]:
task_visit_with_result_str_df['不立刻下单原因主要类别'].fillna('无',inplace=True)
task_visit_with_result_str_df['不立刻下单原因二级分类'].fillna('无',inplace=True)
task_visit_with_result_str_df['客户长时间未下单原因主要类别'].fillna('无',inplace=True)
task_visit_with_result_str_df['客户长时间未下单原因二级分类'].fillna('无',inplace=True)
task_visit_with_result_str_df.to_csv(f"{output_dir}/{task_name}_AI评论结果_合并.csv", index=False)

In [ ]:
summarize_system_prompt = """# 销售拜访记录分析System Prompt

你是一位经验丰富的销售管理分析专家。你的任务是分析用户提供的销售拜访记录CSV数据，并生成一份详细的分析报告。请按照以下结构和要求进行分析：

1. 首先，仔细阅读并解析提供的CSV数据。CSV数据包含以下字段：客户名称、客户业态、拜访时间、客户长时间未下单原因、不愿当场下单原因、是否有效、评论、不立刻下单原因主要类别、不立刻下单原因二级分类、客户长时间未下单原因主要类别、客户长时间未下单原因二级分类。

2. 生成一份标题为"销售拜访记录分析"的Markdown格式报告，包含以下几个主要部分：

## 1. 记录质量
- 统计高质量和低质量记录的数量和百分比
- 指出记录中存在的主要问题
- 提供改进记录质量的建议

## 2. 客户未下单原因分析
- 总结主要的未下单原因
- 提供针对这些原因的建议

## 3. 推荐活动和产品
- 列出记录中提及的产品和活动
- 提供相关的建议以改善产品推荐

## 4. 客户类型分析
- 统计不同客户业态的数量
- 针对客户类型提供相关建议

## 5. 跟进策略
- 根据记录质量和客户情况，提出跟进策略建议

## 6. 销售团队绩效
- 统计有效和无效拜访的数量和百分比
- 提供提升销售团队绩效的建议

## 总结
- 简要总结分析结果和主要建议

3. 在每个部分中，确保：
   - 提供具体的数据和百分比
   - 给出清晰、可执行的建议
   - 使用简洁、专业的语言

4. 分析应基于提供的数据，但也要运用你作为销售管理专家的知识和经验来解读数据并提供洞见。

5. 报告应当客观、深入、实用，能够为销售管理决策提供有价值的参考。

6. 如果某些信息在CSV数据中缺失或不完整，请在相应部分指出，并根据可用信息尽可能进行分析。

请基于这个prompt和用户提供的CSV数据生成分析报告。报告应当结构清晰、内容丰富、见解深刻，能够帮助销售团队改进他们的工作。"""

def call_ai_to_summarize_visit_record(visit_text=""):
    json_text, is_ok = call_azure_openai(
        is_gpt4o=False,
        is_json=False,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": f"{summarize_system_prompt}",
                    }
                ],
            },
            {
                "role": "user",
                "content": [{"type": "text", "text": visit_text}],
            },
        ],
    )
    logging.info(f"json_text:{json_text}, visit_text:{visit_text}")
    return json_text

In [ ]:
# 任务id,任务名称,客户id,客户名称,客户业态,最新归属bd_id,最新归属bd名字,最新m1,最新m2,最新m3,
# 销售区域,实际拜访bd,拜访时间,拜访记录,拜访记录JSON,客户主要采买渠道,客户长时间未下单原因,向客户推荐什么活动及商品,不愿当场下单原因,其他,是否有效,评论,不立刻下单原因主要类别,不立刻下单原因二级分类,客户长时间未下单原因主要类别,客户长时间未下单原因二级分类


for m1_name in task_visit_with_result_str_df["最新m1"].unique():
    m1_df = task_visit_with_result_str_df[
        task_visit_with_result_str_df["最新m1"] == m1_name
    ]
    m2_name = m1_df.iloc[0]["最新m2"]
    file_name = f"{output_dir}/{task_name}_AI评论结果_合并_{m1_name}_{m2_name}.csv"
    m1_df[
        [
            "客户名称",
            "实际拜访bd",
            "最新m1",
            "最新m2",
            "最新m3",
            "销售区域",
            "客户业态",
            "拜访时间",
            "拜访记录",
            "客户长时间未下单原因",
            "向客户推荐什么活动及商品",
            "不愿当场下单原因",
            "是否有效",
            "评论",
            "不立刻下单原因主要类别",
            "不立刻下单原因二级分类",
            "客户长时间未下单原因主要类别",
            "客户长时间未下单原因二级分类",
        ]
    ].to_csv(file_name, index=False)
    csv_text_to_analytic = (
        m1_df[
            [
                "客户名称",
                "客户业态",
                "拜访时间",
                "客户长时间未下单原因",
                "不愿当场下单原因",
                "是否有效",
                "评论",
                "不立刻下单原因主要类别",
                "不立刻下单原因二级分类",
                "客户长时间未下单原因主要类别",
                "客户长时间未下单原因二级分类",
            ]
        ]
        .head(20)
        .to_csv(index=False)
    )
    print(f"已保存文件: {file_name}")
    print(csv_text_to_analytic)

In [ ]:
for bd_name in task_visit_with_result_str_df["实际拜访bd"].unique():
    bd_df = task_visit_with_result_str_df[
        task_visit_with_result_str_df["实际拜访bd"] == bd_name
    ]
    m1_name = bd_df.iloc[0]["最新m1"]
    m2_name = bd_df.iloc[0]["最新m2"]
    file_name = f"{output_dir}/{task_name}_AI评论总结_{bd_name}_{m1_name}_{m2_name}.md"
    csv_text_to_analytic = bd_df[
        [
            "客户名称",
            "客户业态",
            "拜访时间",
            "客户长时间未下单原因",
            "不愿当场下单原因",
            "是否有效",
            "评论",
            "不立刻下单原因主要类别",
            "不立刻下单原因二级分类",
            "客户长时间未下单原因主要类别",
            "客户长时间未下单原因二级分类",
        ]
    ].to_csv(index=False)
    print(f"已保存文件: {file_name}")
    print(csv_text_to_analytic)
    summary_text = call_ai_to_summarize_visit_record(csv_text_to_analytic)
    total_records_count = len(bd_df)
    total_mid_count = len(bd_df["客户id"].unique())
    valid_records_count = len(bd_df[bd_df["是否有效"] == "有效"])
    valid_records_percent = round(valid_records_count / total_records_count * 100, 2)
    summary_text = f"""# 销售{bd_name}的拜访记录总结
    
M1:{m1_name},M2:{m2_name}
        
## 记录质量

- 任务名字: {', '.join(list(bd_df['任务名称'].unique()))}
- 总记录数: {total_records_count}
- 总客户数: {total_mid_count}
- 有效记录数: {valid_records_count}
- 有效记录百分比: {valid_records_percent}%
- 无效记录数: {total_records_count - valid_records_count}
- 无效记录百分比: {100 - valid_records_percent}%
- 拜访记录时间跨度: {bd_df['拜访时间'].min()} ~ {bd_df['拜访时间'].max()}
        
## 客户不愿当场下单原因分析
- 主要类别分布:
 
{bd_df["客户长时间未下单原因主要类别"].value_counts().to_markdown()}

- 二级分类分布: 

{bd_df["客户长时间未下单原因二级分类"].value_counts().to_markdown()}
        
## 不立刻下单原因分析

- 主要原因分布:: 

{bd_df["不立刻下单原因主要类别"].value_counts().to_markdown()}
- 次要原因分布:: 

{bd_df["不立刻下单原因二级分类"].value_counts().to_markdown()}

## AI总结        

{summary_text}"""
    with open(file_name, "w") as f:
        f.write(summary_text)